In [1]:
import numpy as np
import random
import pdb
from functools import cmp_to_key

In [2]:
def load_txt(path):
    f=open(path, "r")
    if(path[-3:] == 'txt'):
        contents =f.readlines()

        dataset = []
        dic = {}
        
        for i in range(len(contents)):
            x = contents[i].split(",")
            for j in range(len(x)):
                if j == 0 :
                    dic.update({'ID_number': x[j]})
                elif j == 1 :
                    dic.update({'Diagnosis': x[j]})
                else :
                    if(j != len(x)-1):
                        dic.update({'features_'+str(j-1): x[j]})
                    else:
                        dic.update({'features_'+str(j-1): x[j][:-1]})
            dataset.append(dic.copy())
                
    return dataset

In [3]:
train = load_txt('./wdbc.data.txt')

In [4]:
train

[{'ID_number': '842302',
  'Diagnosis': 'M',
  'features_1': '17.99',
  'features_2': '10.38',
  'features_3': '122.8',
  'features_4': '1001',
  'features_5': '0.1184',
  'features_6': '0.2776',
  'features_7': '0.3001',
  'features_8': '0.1471',
  'features_9': '0.2419',
  'features_10': '0.07871',
  'features_11': '1.095',
  'features_12': '0.9053',
  'features_13': '8.589',
  'features_14': '153.4',
  'features_15': '0.006399',
  'features_16': '0.04904',
  'features_17': '0.05373',
  'features_18': '0.01587',
  'features_19': '0.03003',
  'features_20': '0.006193',
  'features_21': '25.38',
  'features_22': '17.33',
  'features_23': '184.6',
  'features_24': '2019',
  'features_25': '0.1622',
  'features_26': '0.6656',
  'features_27': '0.7119',
  'features_28': '0.2654',
  'features_29': '0.4601',
  'features_30': '0.1189'},
 {'ID_number': '842517',
  'Diagnosis': 'M',
  'features_1': '20.57',
  'features_2': '17.77',
  'features_3': '132.9',
  'features_4': '1326',
  'features_5

In [5]:
class Chomosome:
    def __init__(self,genes):
        self.gene = genes
        self.fitness = 0

In [6]:
def partition(nums, low, high):

    pivot = nums[(low + high) // 2].fitness
    i = low - 1
    j = high + 1
    
    while True:
        i += 1
        while nums[i].fitness < pivot:
            i += 1
        j -= 1
        while nums[j].fitness > pivot:
            j -= 1
        if i >= j:
            return j

        nums[i], nums[j] = nums[j], nums[i]

def quick_sort(nums):
    # Create a helper function that will be called recursively
    def _quick_sort(items, low, high):
        if low < high:
            # This is the index after the pivot, where our lists are split
            split_index = partition(items, low, high)
            _quick_sort(items, low, split_index)
            _quick_sort(items, split_index + 1, high)

    _quick_sort(nums, 0, len(nums) - 1)
    

In [16]:
class GA :
    
    def __init__(self):
        self.layer = []
        self.initChomosome = []
        self.chomosomelist = []
        self.trainingset = []
        self.weightmin = -1
        self.weightmax = 1

    def getchomosome(self,layer,amount):
        self.layer = layer 
        chromolen = 0
        
        for i in range(1,len(layer)):
            chromolen = chromolen +  (layer[i-1]*layer[i])
            
        for i in range(amount):
            gene = np.zeros(chromolen)
            for j in range(len(gene)):
                gene[j] = self.weightmin + (self.weightmax - self.weightmin)*random.random()
            self.initChomosome.append(Chomosome(gene))
            
    def trainchomosome(self,trainingset,maxgeneration):
        self.trainingset = trainingset
        self.chomosomelist = self.initChomosome.copy()
#         pdb.set_trace()
        self.computefitnessinlist(self.chomosomelist)
        for i in range(maxgeneration):
                selected = self.randomSelect(int(len(self.chomosomelist)*0.9))
                crossed = self.crossover(selected,int(len(self.chomosomelist)*0.9))
                pool = self.chomosomelist.copy()
#                 print(len(pool))
                pool.extend(crossed)
#                 print(len(pool))
                mutateAmount = int(len(self.chomosomelist)*0.4)
                mutated = self.mutate(pool,mutateAmount)
                quick_sort(pool)
                pool = pool[::-1]
#                 pdb.set_trace()
                self.chomosomelist = pool[0:len(self.chomosomelist) - mutateAmount]
                self.chomosomelist.extend(mutated)
                print("Generation : "+str(i+1)+" best accurate : "+str(self.chomosomelist[0].fitness*100/len(trainingset))+"%")
                quick_sort(self.chomosomelist)
                self.chomosomelist = self.chomosomelist[::-1]

    
    def testchomosome(self,testset):
        mlp = MLP(self.layer)
        bestChomosome = self.chomosomelist[0]
        mlp.initweight(bestChomosome)
        bestChomosome.fitness = 0
        for i in testset:
            if(mlp.forward(i)):
                bestChomosome.fitness += 1
        print("Testing accurate: "+str(bestChomosome.fitness*100/len(testset))+"%")
        return 100-(bestChomosome.fitness*100/len(testset))
    
    def computefitness(self,chomosome):
        mlp = MLP(self.layer)
        mlp.initweight(chomosome)
        chomosome.fitness = 0
        for idx,i in enumerate(self.trainingset) :
            if mlp.forward(i) :
                chomosome.fitness += 1
#         print(chomosome.fitness)
        
    def computefitnessinlist(self,chomosomelist):
        j = 0
#         pdb.set_trace()
        for i in chomosomelist:
#             print(str(j) + " : ",end='')
            j = j+1
            self.computefitness(i)
    
    def randomSelect(self,amount):
        selected = []
        for i in range(amount):
            selec = random.randrange(len(self.chomosomelist))
#             print('select : '+str(i))
            selected.append(self.chomosomelist[selec])
        return selected
    
    def crossover(self,selected,amount):
        crossed = []
        for i in range(amount):
            i1 = random.randrange(len(selected))
            i2 = random.randrange(len(selected))
            daddy = selected[i1]
            mommy = selected[i2]
            chomosomelen = len(selected[0].gene)
            halflen = int(chomosomelen/2)
            gene = np.zeros(chomosomelen)
            for j in range(halflen):
                gene[j] = daddy.gene[j]
            for j in range(halflen,chomosomelen):
                gene[j] = mommy.gene[j]
            crossed.append(Chomosome(gene))
        self.computefitnessinlist(crossed)
        return crossed
    
    def mutate(self,pool,amount):
            mutated = []
            index = random.sample(range(len(pool)), amount)
            k = 0
            for i in index:
                gene = pool[i].gene.copy()
                mutateRate = int(len(pool[i].gene)*0.9)
                for j in range(mutateRate):
                    w = random.randrange(len(pool[i].gene))
                    gene[w] = self.weightmin + (self.weightmax - self.weightmin)*random.random()
                    if(gene[w] > self.weightmax):
                        gene[w] = self.weightmax
                    if(gene[w] < self.weightmin):
                        gene[w] = self.weightmin
                mutated.append(Chomosome(gene))
                self.computefitness(mutated[k])
                k = k+1
            return mutated

In [8]:
class MLP:
    def __init__(self,structure_layer):
        self.maxNode = 0
        self.structure = structure_layer
        
        for i in structure_layer:
            if self.maxNode < i :
                self.maxNode = i 
                
        self.weight = np.zeros((len(structure_layer)-1,self.maxNode,self.maxNode))
#         print(self.weight)

    def initweight(self,chomosome):
        count = 0
#         print(self.weight)
        for i in range(len(self.structure)-1):
            for j in range(self.structure[i]):
                for k in range(self.structure[i+1]):
                    self.weight[i][j][k] = chomosome.gene[count]
#                     print(self.weight[i][j][k])
                    count = count+1
#         print(self.weight)
#         pdb.set_trace()
        
    def forward(self,data):
        y = np.zeros((len(self.structure),self.maxNode))
#         print(list(data.values())[2:])
        y[0] = list(data.values())[2:]
        for i in range(1,len(self.structure)):
            for j in range(self.structure[i]):
                v = 0
                for k in range(self.structure[i-1]):
                    v = v + y[i-1][k]*self.weight[i-1][k][j]
                y[i][j] = np.tanh(v)  
#         print("1 :" +str(y[len(self.structure)-1]))
#         print("1 :" +str(y[len(self.structure)-1][0])+" 2 : "+str(y[len(self.structure)-1][1]))
#         print(data['Diagnosis'])
        if(data['Diagnosis'] == 'B'):
            if(y[len(self.structure)-1][0] < y[len(self.structure)-1][1]):
                return True
            else:
                return False
        else:
            if(y[len(self.structure)-1][0] > y[len(self.structure)-1][1]):
                return True
            else:
                return False
               
    def sigmoid(v):
        return 1/(1+np.exp(-v))

In [23]:
genetic = GA()
structure = [30,15,15,2]
genetic.getchomosome(structure,100)
# genetic.trainchomosome(train,5)
eav = 0 
for c in range(10):
    print("------------------------- # Round "+str(c)+' ------------------------- ')
    i = int(c*len(train)*0.1)
    test = train[i:int(i+int(len(train)*0.1))]
    traingset = train.copy()
    del traingset[i:i+int(len(traingset)*0.1)]
    genetic.trainchomosome(traingset,50)
    eav += genetic.testchomosome(test)
print("Error avg : "+str(eav/10)+'%')

------------------------- # Round 0 ------------------------- 
Generation : 1 best accurate : 87.71929824561404%
Generation : 2 best accurate : 91.61793372319688%
Generation : 3 best accurate : 91.81286549707602%
Generation : 4 best accurate : 91.81286549707602%
Generation : 5 best accurate : 91.81286549707602%
Generation : 6 best accurate : 91.81286549707602%
Generation : 7 best accurate : 91.81286549707602%
Generation : 8 best accurate : 91.81286549707602%
Generation : 9 best accurate : 91.81286549707602%
Generation : 10 best accurate : 91.81286549707602%
Generation : 11 best accurate : 91.81286549707602%
Generation : 12 best accurate : 91.81286549707602%
Generation : 13 best accurate : 91.81286549707602%
Generation : 14 best accurate : 91.81286549707602%
Generation : 15 best accurate : 91.81286549707602%
Generation : 16 best accurate : 91.81286549707602%
Generation : 17 best accurate : 91.81286549707602%
Generation : 18 best accurate : 91.81286549707602%
Generation : 19 best accurat

Generation : 7 best accurate : 91.2280701754386%
Generation : 8 best accurate : 91.2280701754386%
Generation : 9 best accurate : 91.2280701754386%
Generation : 10 best accurate : 91.2280701754386%
Generation : 11 best accurate : 91.2280701754386%
Generation : 12 best accurate : 91.2280701754386%
Generation : 13 best accurate : 91.2280701754386%
Generation : 14 best accurate : 92.00779727095517%
Generation : 15 best accurate : 92.00779727095517%
Generation : 16 best accurate : 92.00779727095517%
Generation : 17 best accurate : 92.00779727095517%
Generation : 18 best accurate : 92.98245614035088%
Generation : 19 best accurate : 92.98245614035088%
Generation : 20 best accurate : 92.98245614035088%
Generation : 21 best accurate : 92.98245614035088%
Generation : 22 best accurate : 92.98245614035088%
Generation : 23 best accurate : 92.98245614035088%
Generation : 24 best accurate : 92.98245614035088%
Generation : 25 best accurate : 92.98245614035088%
Generation : 26 best accurate : 92.982456

Generation : 13 best accurate : 90.64327485380117%
Generation : 14 best accurate : 90.64327485380117%
Generation : 15 best accurate : 90.64327485380117%
Generation : 16 best accurate : 90.64327485380117%
Generation : 17 best accurate : 90.64327485380117%
Generation : 18 best accurate : 90.64327485380117%
Generation : 19 best accurate : 91.03313840155946%
Generation : 20 best accurate : 91.03313840155946%
Generation : 21 best accurate : 91.03313840155946%
Generation : 22 best accurate : 91.03313840155946%
Generation : 23 best accurate : 91.03313840155946%
Generation : 24 best accurate : 91.03313840155946%
Generation : 25 best accurate : 91.03313840155946%
Generation : 26 best accurate : 91.42300194931774%
Generation : 27 best accurate : 91.42300194931774%
Generation : 28 best accurate : 91.42300194931774%
Generation : 29 best accurate : 91.42300194931774%
Generation : 30 best accurate : 91.42300194931774%
Generation : 31 best accurate : 91.42300194931774%
Generation : 32 best accurate :

KeyboardInterrupt: 